In [3]:
import pandas as pd
import numpy as np
import pymysql
from sqlalchemy import create_engine

# Load CSV and clean it
data = pd.read_csv(r"C:\\Users\\INDIA\\Downloads\\OLA_DataSet.xlsx - July.csv")

# Rename columns for consistency
data.rename(columns={"Vehicle Images": "Vehicle_Images"}, inplace=True)


#data preprocessing
#handling missing , inconsistent values
# Fill missing values for numerical columns with the mean or median
data['V_TAT'].fillna(data['V_TAT'].mean(), inplace=True)
data['C_TAT'].fillna(data['C_TAT'].mean(), inplace=True)
data['Booking_Value'].fillna(data['Booking_Value'].mean(), inplace=True)
data['Ride_Distance'].fillna(data['Ride_Distance'].mean(), inplace=True)
data['Driver_Ratings'].fillna(data['Driver_Ratings'].mean(), inplace=True)
data['Customer_Rating'].fillna(data['Customer_Rating'].mean(), inplace=True)

# Fill missing values for categorical columns with the mode
data['Canceled_Rides_by_Customer'].fillna('No comment', inplace=True)
data['Canceled_Rides_by_Driver'].fillna('No comment', inplace=True)
data['Incomplete_Rides'].fillna('No', inplace=True)
data['Incomplete_Rides_Reason'].fillna('None', inplace=True)
data['Payment_Method'].fillna('Unknown', inplace=True)

# Convert data types
data['Date'] = pd.to_datetime(data['Date'])
data['Booking_Value'] = data['Booking_Value'].astype(float)
data['Ride_Distance'] = data['Ride_Distance'].astype(float)
data['Driver_Ratings'] = data['Driver_Ratings'].astype(float)
data['Customer_Rating'] = data['Customer_Rating'].astype(float)


C:\Users\INDIA\AppData\Local\Temp\ipykernel_100\3505703740.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['V_TAT'].fillna(data['V_TAT'].mean(), inplace=True)
C:\Users\INDIA\AppData\Local\Temp\ipykernel_100\3505703740.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

F

In [5]:
import pandas as pd
import numpy as np
import pymysql
from sqlalchemy import create_engine

# Establish MySQL connection
mydb = pymysql.connect(user='root', password='Jeelani19', host='127.0.0.1', database="ola")
cursor = mydb.cursor()

# Create table if it doesn't exist
create_table_query = """
CREATE TABLE IF NOT EXISTS ola_data (
    date_ DATETIME,
    time_ TIME,
    Booking_ID VARCHAR(255),
    Booking_Status VARCHAR(255),
    Customer_ID VARCHAR(255),
    Vehicle_Type VARCHAR(255),
    Pickup_Location VARCHAR(255),
    Drop_Location VARCHAR(255),
    V_TAT INT,
    C_TAT INT,
    Canceled_Rides_by_Customer VARCHAR(255),
    Canceled_Rides_by_Driver VARCHAR(255),
    Incomplete_Rides VARCHAR(255),
    Incomplete_Rides_Reason VARCHAR(255),
    Booking_Value FLOAT,
    Payment_Method VARCHAR(255),
    Ride_Distance FLOAT,
    Driver_Ratings FLOAT,
    Customer_Rating FLOAT,
    Vehicle_Images VARCHAR(255)
);
"""
cursor.execute(create_table_query)
mydb.commit()

# Use INSERT IGNORE to skip duplicates
insert_query = """
INSERT IGNORE INTO ola_data (
    date_, time_, Booking_ID, Booking_Status, Customer_ID, Vehicle_Type,
    Pickup_Location, Drop_Location, V_TAT, C_TAT,
    Canceled_Rides_by_Customer, Canceled_Rides_by_Driver,
    Incomplete_Rides, Incomplete_Rides_Reason,
    Booking_Value, Payment_Method, Ride_Distance,
    Driver_Ratings, Customer_Rating, Vehicle_Images
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# Insert row by row
for index, row in data.iterrows():
    try:
        values = (
            row["Date"],
            row["Time"],
            row["Booking_ID"],
            row["Booking_Status"],
            row["Customer_ID"],
            row["Vehicle_Type"],
            row["Pickup_Location"],
            row["Drop_Location"],
            float(row["V_TAT"]) if row["V_TAT"] is not None else None,
            float(row["C_TAT"]) if row["C_TAT"] is not None else None,
            row["Canceled_Rides_by_Customer"],
            row["Canceled_Rides_by_Driver"],
            row["Incomplete_Rides"],
            row["Incomplete_Rides_Reason"],
            float(row["Booking_Value"]) if row["Booking_Value"] is not None else None,
            row["Payment_Method"],
            int(row["Ride_Distance"]) if row["Ride_Distance"] is not None else None,
            float(row["Driver_Ratings"]) if row["Driver_Ratings"] is not None else None,
            float(row["Customer_Rating"]) if row["Customer_Rating"] is not None else None,
            row["Vehicle_Images"]
        )
        cursor.execute(insert_query, values)
        mydb.commit()

    except Exception as e:
        print(f"Error at row {index}: {e}")
        print(f"Problematic data: {row.to_dict()}")

# Commit the changes
mydb.commit()

In [29]:
import pandas as pd
import numpy as np
import pymysql
from sqlalchemy import create_engine

# Establish MySQL connection
mydb = pymysql.connect(user='root', password='Jeelani19', host='127.0.0.1', database="ola")
cursor = mydb.cursor()

# SQL Queries to answer questions

#1. Retrieve all successful bookings:
cursor.execute(" SELECT * from ola_data WHERE Booking_Status= 'Success' \n")
successful_rides = cursor.fetchone() # fetches the first line of row
print(f"Number of successful rides: {successful_rides}")

#2. Find the average ride distance for each vehicle type:
cursor.execute(" SELECT Vehicle_Type, AVG(Ride_Distance) AS Average_Ride_Distance FROM ola_data WHERE Booking_Status = 'Success' GROUP BY Vehicle_Type \n")
average_distance = cursor.fetchone() # fetches the first line of row
print(f"average ride distance for each vehicle type: {average_distance}")

#3.Get the total number of cancelled rides by customers:
cursor.execute("SELECT COUNT(*) AS Total_Cancelled_Rides_By_Customers FROM ola_data WHERE Booking_Status = 'Canceled by Customer' \n")
total_cancelled = cursor.fetchone() 
print(f"total_cancelled by customers: {total_cancelled}")

#4.List the top 5 customers who booked the highest number of rides:
cursor.execute("SELECT Customer_ID, COUNT(*) AS Number_Of_Rides FROM ola_data GROUP BY Customer_ID ORDER BY Number_Of_Rides DESC LIMIT 5 \n")
top_5 = cursor.fetchone() 
print(f"top 5 ride bookings by customers: {top_5}")

#5. Get the number of rides cancelled by drivers due to personal and car-related issues:
cursor.execute("SELECT COUNT(*) AS Total_Cancelled_Rides_By_Drivers FROM ola_data WHERE Booking_Status = 'Canceled by Driver' AND Incomplete_Rides_Reason LIKE '%Personal & Car related issue%' \n")
canceled_rides = cursor.fetchone() 
print(f"cancelled rides by due to personal and car-related issues: {canceled_rides}")

#6. Find the maximum and minimum driver ratings for Prime Sedan bookings:
cursor.execute("SELECT MAX(Driver_Ratings) AS Max_Driver_Rating, MIN(Driver_Ratings) AS Min_Driver_Rating FROM ola_data WHERE Vehicle_Type = 'Prime Sedan' AND Booking_Status = 'Success' \n")
max_min = cursor.fetchone() 
print(f"maximum and minimum driver ratings for Prime Sedan: {max_min}")

#7. Retrieve all rides where payment was made using UPI:
cursor.execute("SELECT * FROM ola_data WHERE Payment_Method = 'UPI' \n")
UPI = cursor.fetchone() 
print(f"payments by UPI: {UPI}")

#8. Find the average customer rating per vehicle type:
cursor.execute("SELECT Vehicle_Type, AVG(Customer_Rating) AS Average_Customer_Rating FROM ola_data WHERE Booking_Status = 'Success' GROUP BY Vehicle_Type \n")
avg_customer = cursor.fetchone() 
print(f"Avg customer rating by vehicle: {avg_customer}")

#9. Calculate the total booking value of rides completed successfully:
cursor.execute("SELECT SUM(Booking_Value) AS Total_Booking_Value FROM ola_data WHERE Booking_Status = 'Success' \n")
total_successful = cursor.fetchone() 
print(f"total_successful rides: {total_successful}")

#10. List all incomplete rides along with the reason:
cursor.execute("SELECT Incomplete_Rides, Incomplete_Rides_Reason FROM ola_data WHERE Incomplete_Rides = 'Yes'  \n")
total_successful = cursor.fetchone() 
print(f"incomplete rides: {total_successful}")

Number of successful rides: (datetime.datetime(2024, 7, 25, 22, 20), datetime.timedelta(seconds=80400), 'CNR2940424040', 'Success', 'CID225428', 'Bike', 'Magadi Road', 'Varthur', 203, 30, 'No comment', 'No comment', 'No', 'None', 158.0, 'Cash', 13.0, 4.1, 4.0, 'https://cdn-icons-png.flaticon.com/128/9983/9983173.png', 2)
average ride distance for each vehicle type: ('Bike', 24.9338734398949)
total_cancelled by customers: (10499,)
top 5 ride bookings by customers: ('CID954071', 5)
cancelled rides by due to personal and car-related issues: (0,)
maximum and minimum driver ratings for Prime Sedan: (5.0, 3.0)
payments by UPI: (datetime.datetime(2024, 7, 30, 19, 59), datetime.timedelta(seconds=71940), 'CNR2982357879', 'Success', 'CID270156', 'Prime SUV', 'Sahakar Nagar', 'Varthur', 238, 130, 'No comment', 'No comment', 'No', 'None', 386.0, 'UPI', 40.0, 4.2, 4.8, 'https://cdn-icons-png.flaticon.com/128/9983/9983204.png', 2)
Avg customer rating by vehicle: ('Bike', 3.9933763974496514)
total_su